# Part 1

In [1]:
filename = "inputs/12-19.txt"
with open(filename, "r") as f:
    data = f.read()

In [2]:
import re
from functools import cache
import math

In [3]:
blueprints = {}
for l in data.splitlines():
    bp, or_or, cl_or, ob_or, ob_cl, ge_or, ge_ob = (int(v) for v in re.findall("\d+", l))
    blueprints[bp] = {
        "ore": {
            "ore": or_or,
        },
        "clay": {
            "ore": cl_or,
        },
        "obsidian": {
            "ore": ob_or,
            "clay": ob_cl,
        },
        "geode": {
            "ore": ge_or,
            "obsidian": ge_ob,
        },
    }
def bp_to_tuple(bp):
    return tuple(v for d in bp.values() for v in d.values())

In [4]:
@cache
def geodes(t, state, bp):
    (ore_r, clay_r, obs_r, geo_r, ore, clay, obs, geo) = state
    (ore_ore, clay_ore, obs_ore, obs_clay, geo_ore, geo_obs) = bp
    # stop if we can't possibly build another geode before t = 1 (excluded)
    if obs + (obs_r + (clay + (clay_r + (ore + ore_r * (t-4))//clay_ore) * (t-3))//obs_clay) * (t-2) < geo_obs:
        return geo + t * geo_r
    else:
        # build a geode robot now if possible, as delaying geode robots is never profitable
        if ore >= geo_ore and obs >= geo_obs:
            return geodes(
                t - 1,
                (ore_r, clay_r, obs_r, geo_r + 1, ore + ore_r - geo_ore, clay + clay_r, obs + obs_r - geo_obs, geo + geo_r),
                bp
            )
        geos = [0]
        # build a geode robot
        if obs_r > 0:
            dt = max((math.ceil((geo_obs - obs) / obs_r), math.ceil((geo_ore - ore) / ore_r), 0)) + 1
            if t - dt >= 1:
                geos.append(geodes(
                    t - dt,
                    (ore_r, clay_r, obs_r, geo_r + 1, ore + dt * ore_r - geo_ore, clay + dt * clay_r, obs + dt * obs_r - geo_obs, geo + dt * geo_r),
                    bp
                ))
        # build an obsidian robot (up to a maximum)
        if clay_r > 0 and obs_r < geo_obs:
            dt = max((math.ceil((obs_clay - clay) / clay_r), math.ceil((obs_ore - ore) / ore_r), 0)) + 1
            if t - dt >= 2:
                geos.append(geodes(
                    t - dt,
                    (ore_r, clay_r, obs_r + 1, geo_r, ore + dt * ore_r - obs_ore, clay + dt * clay_r - obs_clay, obs + dt * obs_r, geo + dt * geo_r),
                    bp
                ))
        # build a clay robot (up to a maximum)
        if clay_r < obs_clay:
            dt = max(math.ceil((clay_ore - ore) / ore_r), 0) + 1
            if t - dt >= 3:
                geos.append(geodes(
                    t - dt,
                    (ore_r, clay_r + 1, obs_r, geo_r, ore + dt * ore_r - clay_ore, clay + dt * clay_r, obs + dt * obs_r, geo + dt * geo_r),
                    bp
                ))
        # build an ore robot (up to a maximum)
        if ore_r < max([clay_ore, obs_ore, geo_ore]):
            dt = max(math.ceil((ore_ore - ore) / ore_r), 0) + 1
            if t - dt >= 2:
                geos.append(geodes(
                    t - dt,
                    (ore_r + 1, clay_r, obs_r, geo_r, ore + dt * ore_r - ore_ore, clay + dt * clay_r, obs + dt * obs_r, geo + dt * geo_r),
                    bp
                ))

        return max(geos)

In [5]:
%%time
s = 0
for i, bp in blueprints.items():
    geo = geodes(24, (1, 0, 0, 0, 0, 0, 0, 0), bp_to_tuple(bp))
    s += i * geo
s

CPU times: total: 344 ms
Wall time: 336 ms


1092

# Part 2

In [6]:
%%time
p = 1
for i, bp in list(blueprints.items())[:3]:
    geo = geodes(32, (1, 0, 0, 0, 0, 0, 0, 0), bp_to_tuple(bp))
    p *= geo
p

CPU times: total: 3.19 s
Wall time: 3.21 s


3542